# Reading and Processing Data from STAC

We will query a STAC catalog for Sentinel-2 imagery and load a single image using XArray. We will also apply some data processing to clip the image and calculate spectral indices.

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install pystac-client
    !apt install libspatialindex-dev
    !pip install fiona shapely pyproj rtree
    !pip install geopandas folium stackstac rioxarray mapclassify

In [ ]:
import json
import geopandas as gpd
from shapely.geometry import mapping
import pandas as pd
from pystac_client import Client
import os
import folium
from folium import Figure
import stackstac
import rioxarray
import matplotlib.pyplot as plt
import mapclassify

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' +
         'bangalore.geojson')

## Procedure

Let's use Element84 search endpoint to look for items from the sentinel-2-l2a collection on AWS

In [ ]:
catalog = Client.open('https://earth-search.aws.element84.com/v1')

In [ ]:
aoi_file = 'bangalore.geojson'
aoi_filepath = os.path.join(data_folder, aoi_file)
aoi = gpd.read_file(aoi_filepath)

In [ ]:
geometry = aoi.unary_union
geometry_geojson = json.dumps(mapping(geometry))

In [ ]:
time_range = "2023-02-01/2023-02-28"

search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=geometry_geojson,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 30}},
)
items = search.item_collection()
len(items)

In [ ]:
items_df = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')
items_df

In [ ]:
fig = Figure(width=800, height=400)
m = folium.Map()
bounds = items_df.total_bounds
m.fit_bounds([[bounds[1],bounds[0]], [bounds[3],bounds[2]]])

items_df.explore(m=m,
                 color='black',
                 tooltip=['created'],
                 style_kwds={'fillOpacity': 0.2, 'weight': 0.5},)
aoi.explore(m=m, color='blue')
fig.add_child(m)


In [ ]:
stack = stackstac.stack(items)
stack

In [ ]:
scene = stack.isel(time=6).sel(band=['red', 'green', 'blue', 'nir', 'swir16'])
scene

In [ ]:
geometry = aoi.to_crs(scene.rio.crs).geometry
clipped = scene.rio.clip(geometry)
clipped

In [ ]:
scaled = (clipped - 1000)/10000

In [ ]:
%time scaled = scaled.compute()

In [ ]:
rgb = scaled.sel(band=['red', 'green', 'blue'])

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
rgb.plot.imshow(
    ax=ax,
    robust=True)
ax.set_title('RGB Visualization')
ax.set_axis_off()
plt.show()

In [ ]:
red = scaled.sel(band='red')
nir = scaled.sel(band='nir')
green = scaled.sel(band='green')
swir1 = scaled.sel(band='swir16')

mndwi = (green - swir1)/(green + swir1)
ndvi = (nir - red)/(nir + red)
ndbi = (swir1 - nir)/(swir1 + nir)

In [ ]:
%%time
files = {
    'rgb.tif': rgb,
    'ndvi.tif': ndvi,
    'mndwi.tif': mndwi,
    'ndbi.rif': ndbi

}

for file in files:
  output_path = os.path.join(output_folder, file)
  files[file].rio.to_raster(output_path, driver='COG')
  print(f'Wrote {file}')